# Target Guided Ordinal Encoding

Target Guided Ordinal Encoding (also called Target Encoding or Mean Encoding) is an advanced encoding technique that uses the **relationship between a categorical feature and the target variable** to assign numerical values.

## What is Target Guided Ordinal Encoding?

Instead of manually defining the order of categories (like in Ordinal Encoding), Target Encoding automatically creates an order based on the **target variable's statistics** (usually the mean) for each category.

### How It Works

1. For each category in a feature, calculate the mean of the target variable
2. Order categories based on these means
3. Assign ordinal values (0, 1, 2, ...) based on this ordering

### Example

Suppose you have a "City" feature and want to predict "Salary":

| City | Average Salary |
|------|---------------|
| New York | $120,000 |
| San Francisco | $115,000 |
| Boston | $95,000 |
| Chicago | $85,000 |

Target Encoding would assign: Chicago → 0, Boston → 1, San Francisco → 2, New York → 3

## Advantages

- **Captures predictive power**: Categories are ordered by their relationship with the target
- **Works for high cardinality**: Handles many categories efficiently
- **Improves model performance**: Better than arbitrary ordering

## Disadvantages

- **Overfitting risk**: Model may memorize training data patterns
- **Requires target variable**: Cannot be used for encoding test data without training reference
- **Needs careful cross-validation**: Must prevent data leakage

## When to Use

✅ **Use Target Encoding when:**
- You have high cardinality categorical features (many unique values)
- Categories don't have a natural order
- You want to capture the predictive relationship between feature and target
- Working with tree-based models or linear models

❌ **Avoid when:**
- You have very few samples per category (unreliable means)
- Your dataset is very small (high overfitting risk)
- You need interpretability (relationship is indirect)

## Step 1: Create Sample Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

# Set random seed for reproducibility
np.random.seed(42)

# Create a dataset with City and Department (high cardinality features)
n_samples = 300

cities = np.random.choice(['New York', 'San Francisco', 'Boston', 'Chicago', 'Seattle', 
                           'Austin', 'Denver', 'Portland', 'Atlanta', 'Miami'], 
                          n_samples, p=[0.15, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05])

departments = np.random.choice(['Engineering', 'Sales', 'Marketing', 'HR', 'Finance', 
                                'Operations', 'Customer Support'], 
                               n_samples, p=[0.25, 0.2, 0.15, 0.1, 0.1, 0.1, 0.1])

experience = np.random.randint(0, 20, n_samples)

# Define salary based on city and department (hidden pattern)
city_salary = {
    'New York': 120000, 'San Francisco': 115000, 'Boston': 95000, 
    'Chicago': 85000, 'Seattle': 100000, 'Austin': 90000, 
    'Denver': 85000, 'Portland': 88000, 'Atlanta': 82000, 'Miami': 80000
}

department_salary = {
    'Engineering': 110000, 'Sales': 90000, 'Marketing': 85000, 
    'HR': 75000, 'Finance': 95000, 'Operations': 80000, 'Customer Support': 70000
}

# Calculate salary with some noise
salaries = []
for city, dept, exp in zip(cities, departments, experience):
    base_salary = (city_salary[city] + department_salary[dept]) / 2
    experience_bonus = exp * 2000
    noise = np.random.normal(0, 5000)
    salaries.append(base_salary + experience_bonus + noise)

# Create DataFrame
df = pd.DataFrame({
    'City': cities,
    'Department': departments,
    'Experience': experience,
    'Salary': salaries
})

print("="*70)
print("DATASET OVERVIEW")
print("="*70)
print(f"\nDataset Shape: {df.shape}")
print(f"Number of unique cities: {df['City'].nunique()}")
print(f"Number of unique departments: {df['Department'].nunique()}")

print("\n" + "-"*70)
print("First 10 rows:")
print("-"*70)
print(df.head(10))

print("\n" + "-"*70)
print("Statistical Summary:")
print("-"*70)
print(df.describe())

# Visualize the distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Dataset Exploration', fontsize=16, fontweight='bold')

# City distribution
city_counts = df['City'].value_counts()
axes[0, 0].barh(city_counts.index, city_counts.values, color='skyblue')
axes[0, 0].set_xlabel('Count')
axes[0, 0].set_title('City Distribution', fontweight='bold')
axes[0, 0].grid(axis='x', alpha=0.3)

# Department distribution
dept_counts = df['Department'].value_counts()
axes[0, 1].barh(dept_counts.index, dept_counts.values, color='lightcoral')
axes[0, 1].set_xlabel('Count')
axes[0, 1].set_title('Department Distribution', fontweight='bold')
axes[0, 1].grid(axis='x', alpha=0.3)

# Experience distribution
axes[1, 0].hist(df['Experience'], bins=20, color='lightgreen', edgecolor='black')
axes[1, 0].set_xlabel('Years of Experience')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Experience Distribution', fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# Salary distribution
axes[1, 1].hist(df['Salary'], bins=30, color='orange', edgecolor='black')
axes[1, 1].set_xlabel('Salary ($)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Salary Distribution (Target Variable)', fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("✅ Dataset created successfully!")
print("="*70)

## Step 2: Calculate Target Statistics for Each Category

Calculate the mean salary for each city and department to understand the relationship.

In [ ]:
# Calculate mean salary for each category
print("="*70)
print("TARGET STATISTICS (Mean Salary by Category)")
print("="*70)

# City - Mean Salary
city_means = df.groupby('City')['Salary'].mean().sort_values(ascending=False)
print("\n1. Average Salary by City (Sorted High to Low):")
print("-"*70)
for city, salary in city_means.items():
    print(f"   {city:20s} → ${salary:,.2f}")

# Department - Mean Salary
department_means = df.groupby('Department')['Salary'].mean().sort_values(ascending=False)
print("\n2. Average Salary by Department (Sorted High to Low):")
print("-"*70)
for dept, salary in department_means.items():
    print(f"   {dept:20s} → ${salary:,.2f}")

# Visualize target statistics
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Mean Salary by Category (Target Statistics)', fontsize=16, fontweight='bold')

# City
axes[0].barh(city_means.index, city_means.values, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Average Salary ($)')
axes[0].set_title('Average Salary by City', fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)
for i, (city, salary) in enumerate(city_means.items()):
    axes[0].text(salary + 1000, i, f'${salary:,.0f}', va='center', fontsize=9)

# Department
axes[1].barh(department_means.index, department_means.values, color='lightcoral', edgecolor='black')
axes[1].set_xlabel('Average Salary ($)')
axes[1].set_title('Average Salary by Department', fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)
for i, (dept, salary) in enumerate(department_means.items()):
    axes[1].text(salary + 1000, i, f'${salary:,.0f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("📊 KEY INSIGHT:")
print("="*70)
print("These mean salaries will be used to create the ordinal encoding order.")
print("Higher mean salary → Higher ordinal value")
print("This captures the predictive relationship between category and target!")

## Step 3: Implement Target Guided Ordinal Encoding

Now we'll encode categories based on their mean target values.

In [ ]:
# Method 1: Simple Target Encoding (using mean values directly)
# This maps each category to its mean target value

def target_guided_encoding(df, column, target):
    """
    Encode categorical feature using target mean
    
    Parameters:
    - df: DataFrame
    - column: Categorical column to encode
    - target: Target variable column
    
    Returns:
    - Encoding dictionary
    """
    # Calculate mean of target for each category
    encoding_dict = df.groupby(column)[target].mean().to_dict()
    return encoding_dict

# Split data first to prevent data leakage
X = df[['City', 'Department', 'Experience']]
y = df['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create training DataFrame for encoding
train_df = X_train.copy()
train_df['Salary'] = y_train

# Calculate target encoding on TRAINING data only
city_encoding = target_guided_encoding(train_df, 'City', 'Salary')
department_encoding = target_guided_encoding(train_df, 'Department', 'Salary')

print("="*70)
print("TARGET GUIDED ENCODING DICTIONARIES")
print("="*70)

print("\n1. City Encoding (Mean Salary):")
print("-"*70)
for city, value in sorted(city_encoding.items(), key=lambda x: x[1], reverse=True):
    print(f"   {city:20s} → {value:,.2f}")

print("\n2. Department Encoding (Mean Salary):")
print("-"*70)
for dept, value in sorted(department_encoding.items(), key=lambda x: x[1], reverse=True):
    print(f"   {dept:20s} → {value:,.2f}")

# Apply encoding to train and test sets
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

# For training data
X_train_encoded['City_TargetEncoded'] = X_train_encoded['City'].map(city_encoding)
X_train_encoded['Department_TargetEncoded'] = X_train_encoded['Department'].map(department_encoding)

# For test data (use training encodings)
X_test_encoded['City_TargetEncoded'] = X_test_encoded['City'].map(city_encoding)
X_test_encoded['Department_TargetEncoded'] = X_test_encoded['Department'].map(department_encoding)

# Handle unknown categories in test set (if any) - use global mean
global_mean = y_train.mean()
X_test_encoded['City_TargetEncoded'].fillna(global_mean, inplace=True)
X_test_encoded['Department_TargetEncoded'].fillna(global_mean, inplace=True)

print("\n" + "-"*70)
print("Encoded Training Data (First 10 rows):")
print("-"*70)
print(X_train_encoded[['City', 'City_TargetEncoded', 'Department', 
                        'Department_TargetEncoded', 'Experience']].head(10))

print("\n" + "="*70)
print("✅ Target Guided Encoding completed!")
print("="*70)
print("Note: Encoding was calculated on TRAINING data only to prevent data leakage.")

## Step 4: Compare with Label Encoding

Let's compare Target Encoding with standard Label Encoding.

In [ ]:
# Compare Label Encoding vs Target Guided Encoding

# Label Encoding
le_city = LabelEncoder()
le_department = LabelEncoder()

X_train_label = X_train.copy()
X_test_label = X_test.copy()

X_train_label['City_LabelEncoded'] = le_city.fit_transform(X_train_label['City'])
X_train_label['Department_LabelEncoded'] = le_department.fit_transform(X_train_label['Department'])

X_test_label['City_LabelEncoded'] = le_city.transform(X_test_label['City'])
X_test_label['Department_LabelEncoded'] = le_department.transform(X_test_label['Department'])

# Train models
print("="*70)
print("MODEL COMPARISON: LABEL ENCODING vs TARGET ENCODING")
print("="*70)

# Model 1: Label Encoding
X_train_label_features = X_train_label[['City_LabelEncoded', 'Department_LabelEncoded', 'Experience']]
X_test_label_features = X_test_label[['City_LabelEncoded', 'Department_LabelEncoded', 'Experience']]

model_label = LinearRegression()
model_label.fit(X_train_label_features, y_train)
y_pred_label = model_label.predict(X_test_label_features)

# Model 2: Target Encoding
X_train_target_features = X_train_encoded[['City_TargetEncoded', 'Department_TargetEncoded', 'Experience']]
X_test_target_features = X_test_encoded[['City_TargetEncoded', 'Department_TargetEncoded', 'Experience']]

model_target = LinearRegression()
model_target.fit(X_train_target_features, y_train)
y_pred_target = model_target.predict(X_test_target_features)

# Compare performance
print("\n1. Model with LABEL ENCODING:")
r2_label = r2_score(y_test, y_pred_label)
mae_label = mean_absolute_error(y_test, y_pred_label)
rmse_label = np.sqrt(mean_squared_error(y_test, y_pred_label))

print(f"   R² Score:  {r2_label:.4f}")
print(f"   MAE:       ${mae_label:,.2f}")
print(f"   RMSE:      ${rmse_label:,.2f}")

print("\n2. Model with TARGET GUIDED ENCODING:")
r2_target = r2_score(y_test, y_pred_target)
mae_target = mean_absolute_error(y_test, y_pred_target)
rmse_target = np.sqrt(mean_squared_error(y_test, y_pred_target))

print(f"   R² Score:  {r2_target:.4f}")
print(f"   MAE:       ${mae_target:,.2f}")
print(f"   RMSE:      ${rmse_target:,.2f}")

improvement_r2 = ((r2_target - r2_label) / r2_label * 100)
improvement_mae = ((mae_label - mae_target) / mae_label * 100)

print(f"\n✅ Improvement with Target Encoding:")
print(f"   R² Score improvement: {improvement_r2:.2f}%")
print(f"   MAE reduction: {improvement_mae:.2f}%")

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Prediction Accuracy: Label Encoding vs Target Encoding', fontsize=16, fontweight='bold')

# Label Encoding predictions
axes[0].scatter(y_test, y_pred_label, alpha=0.5, color='salmon')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Salary ($)')
axes[0].set_ylabel('Predicted Salary ($)')
axes[0].set_title(f'Label Encoding\nR² = {r2_label:.4f}, MAE = ${mae_label:,.0f}', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Target Encoding predictions
axes[1].scatter(y_test, y_pred_target, alpha=0.5, color='lightgreen')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Salary ($)')
axes[1].set_ylabel('Predicted Salary ($)')
axes[1].set_title(f'Target Encoding\nR² = {r2_target:.4f}, MAE = ${mae_target:,.0f}', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Show encoding comparison for a few cities
print("\n" + "="*70)
print("ENCODING COMPARISON EXAMPLE")
print("="*70)
print("\n{:20s} {:20s} {:20s}".format("City", "Label Encoding", "Target Encoding"))
print("-"*70)

city_label_mapping = dict(zip(le_city.classes_, le_city.transform(le_city.classes_)))
for city in ['New York', 'Chicago', 'Miami', 'San Francisco']:
    if city in city_label_mapping and city in city_encoding:
        print("{:20s} {:20d} ${:,.2f}".format(
            city, 
            city_label_mapping[city],
            city_encoding[city]
        ))

print("\n" + "="*70)
print("📊 KEY INSIGHT:")
print("="*70)
print("Target Encoding captures the relationship between city/department and salary,")
print("leading to significantly better predictions. Label Encoding uses arbitrary")
print("alphabetical order, which has no predictive value.")

## Step 5: Handling Overfitting with Smoothing

Target Encoding can overfit, especially with small samples. We can use **smoothing** to regularize the encoding.

In [ ]:
# Smoothed Target Encoding
# Formula: Smoothed_Mean = (n * category_mean + m * global_mean) / (n + m)
# where n = number of samples in category, m = smoothing parameter

def smoothed_target_encoding(df, column, target, m=10):
    """
    Encode categorical feature using smoothed target mean
    
    Parameters:
    - df: DataFrame
    - column: Categorical column to encode
    - target: Target variable column
    - m: Smoothing parameter (higher = more smoothing)
    
    Returns:
    - Encoding dictionary
    """
    # Global mean
    global_mean = df[target].mean()
    
    # Calculate statistics for each category
    agg = df.groupby(column)[target].agg(['mean', 'count'])
    
    # Apply smoothing formula
    smoothed = (agg['count'] * agg['mean'] + m * global_mean) / (agg['count'] + m)
    
    return smoothed.to_dict()

# Apply smoothed encoding
city_smoothed = smoothed_target_encoding(train_df, 'City', 'Salary', m=10)
department_smoothed = smoothed_target_encoding(train_df, 'Department', 'Salary', m=10)

# Compare regular vs smoothed encoding
print("="*70)
print("REGULAR vs SMOOTHED TARGET ENCODING")
print("="*70)

print("\n1. City Encoding Comparison:")
print("-"*70)
print("{:20s} {:>20s} {:>20s}".format("City", "Regular", "Smoothed"))
print("-"*70)

for city in city_encoding.keys():
    regular_val = city_encoding[city]
    smoothed_val = city_smoothed.get(city, 0)
    print("{:20s} ${:>19,.2f} ${:>19,.2f}".format(city, regular_val, smoothed_val))

print("\n2. Department Encoding Comparison:")
print("-"*70)
print("{:20s} {:>20s} {:>20s}".format("Department", "Regular", "Smoothed"))
print("-"*70)

for dept in department_encoding.keys():
    regular_val = department_encoding[dept]
    smoothed_val = department_smoothed.get(dept, 0)
    print("{:20s} ${:>19,.2f} ${:>19,.2f}".format(dept, regular_val, smoothed_val))

# Apply smoothed encoding to datasets
X_train_smoothed = X_train.copy()
X_test_smoothed = X_test.copy()

X_train_smoothed['City_Smoothed'] = X_train_smoothed['City'].map(city_smoothed)
X_train_smoothed['Department_Smoothed'] = X_train_smoothed['Department'].map(department_smoothed)

X_test_smoothed['City_Smoothed'] = X_test_smoothed['City'].map(city_smoothed)
X_test_smoothed['Department_Smoothed'] = X_test_smoothed['Department'].map(department_smoothed)

# Handle missing values
X_test_smoothed['City_Smoothed'].fillna(global_mean, inplace=True)
X_test_smoothed['Department_Smoothed'].fillna(global_mean, inplace=True)

# Train model with smoothed encoding
X_train_smoothed_features = X_train_smoothed[['City_Smoothed', 'Department_Smoothed', 'Experience']]
X_test_smoothed_features = X_test_smoothed[['City_Smoothed', 'Department_Smoothed', 'Experience']]

model_smoothed = LinearRegression()
model_smoothed.fit(X_train_smoothed_features, y_train)
y_pred_smoothed = model_smoothed.predict(X_test_smoothed_features)

# Compare all three methods
print("\n" + "="*70)
print("COMPLETE COMPARISON: LABEL vs TARGET vs SMOOTHED TARGET")
print("="*70)

print("\n{:30s} {:>12s} {:>12s} {:>12s}".format("Metric", "Label", "Target", "Smoothed"))
print("-"*70)

print("{:30s} {:>12.4f} {:>12.4f} {:>12.4f}".format(
    "R² Score", r2_label, r2_target, r2_score(y_test, y_pred_smoothed)))
print("{:30s} ${:>11,.2f} ${:>11,.2f} ${:>11,.2f}".format(
    "MAE", mae_label, mae_target, mean_absolute_error(y_test, y_pred_smoothed)))
print("{:30s} ${:>11,.2f} ${:>11,.2f} ${:>11,.2f}".format(
    "RMSE", rmse_label, rmse_target, np.sqrt(mean_squared_error(y_test, y_pred_smoothed))))

# Visualize effect of smoothing
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Effect of Smoothing on Target Encoding', fontsize=16, fontweight='bold')

# City comparison
cities_sorted = sorted(city_encoding.keys(), key=lambda x: city_encoding[x], reverse=True)
regular_values = [city_encoding[c] for c in cities_sorted]
smoothed_values = [city_smoothed.get(c, 0) for c in cities_sorted]

x_pos = np.arange(len(cities_sorted))
width = 0.35

axes[0].barh(x_pos - width/2, regular_values, width, label='Regular', color='skyblue', alpha=0.8)
axes[0].barh(x_pos + width/2, smoothed_values, width, label='Smoothed', color='lightgreen', alpha=0.8)
axes[0].set_yticks(x_pos)
axes[0].set_yticklabels(cities_sorted)
axes[0].set_xlabel('Encoded Value ($)')
axes[0].set_title('City Encoding: Regular vs Smoothed', fontweight='bold')
axes[0].legend()
axes[0].grid(axis='x', alpha=0.3)

# Department comparison
depts_sorted = sorted(department_encoding.keys(), key=lambda x: department_encoding[x], reverse=True)
regular_values_dept = [department_encoding[d] for d in depts_sorted]
smoothed_values_dept = [department_smoothed.get(d, 0) for d in depts_sorted]

x_pos_dept = np.arange(len(depts_sorted))

axes[1].barh(x_pos_dept - width/2, regular_values_dept, width, label='Regular', color='lightcoral', alpha=0.8)
axes[1].barh(x_pos_dept + width/2, smoothed_values_dept, width, label='Smoothed', color='orange', alpha=0.8)
axes[1].set_yticks(x_pos_dept)
axes[1].set_yticklabels(depts_sorted)
axes[1].set_xlabel('Encoded Value ($)')
axes[1].set_title('Department Encoding: Regular vs Smoothed', fontweight='bold')
axes[1].legend()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("📊 SMOOTHING EFFECT:")
print("="*70)
print("Smoothing reduces extreme values and pulls them toward the global mean.")
print("This helps prevent overfitting, especially for categories with few samples.")
print("The parameter 'm' controls the strength: higher m = more smoothing.")

## Best Practices for Target Guided Encoding

### 1. **Prevent Data Leakage**
- Always calculate encoding on **training data only**
- Apply the same encoding to test/validation data
- Never use test set information in encoding

### 2. **Use Cross-Validation**
- For more robust encoding, use K-Fold cross-validation
- Calculate encoding separately for each fold
- This reduces overfitting risk

### 3. **Apply Smoothing**
- Use smoothing for categories with few samples
- Start with m=10 and tune based on validation performance
- Higher smoothing for smaller datasets

### 4. **Handle Unknown Categories**
- New categories in test data should use global mean
- Consider using `df.map()` with `fillna(global_mean)`

### 5. **When to Use Target Encoding**

| Scenario | Recommended? | Reason |
|----------|-------------|--------|
| High cardinality features (50+ categories) | ✅ Yes | Better than OHE, captures patterns |
| Small dataset (< 1000 samples) | ⚠️ Cautious | Use heavy smoothing or avoid |
| Tree-based models (Random Forest, XGBoost) | ✅ Yes | Works very well |
| Linear models | ✅ Yes | Captures non-linear relationships |
| Features with no target relationship | ❌ No | Use label encoding or OHE |
| Features with clear ordinal order | ❌ No | Use ordinal encoding instead |

### 6. **Comparison Summary**

| Method | Best For | Pros | Cons |
|--------|----------|------|------|
| **Label Encoding** | Binary features, tree models | Simple, fast | Random ordering |
| **Ordinal Encoding** | Ordered categories | Preserves order | Requires domain knowledge |
| **Target Encoding** | High cardinality | Uses target info | Overfitting risk |
| **One-Hot Encoding** | Low cardinality nominal | No assumptions | High dimensionality |

## Summary

### Key Takeaways

1. **Target Guided Ordinal Encoding**:
   - Uses target variable statistics (mean) to encode categories
   - Automatically captures predictive relationships
   - Significantly better than arbitrary label encoding

2. **How It Works**:
   - Calculate mean target value for each category
   - Map categories to these mean values
   - Categories with higher target means get higher encoded values

3. **Critical Requirements**:
   - **Calculate on training data only** (prevent data leakage)
   - Use global mean for unknown test categories
   - Apply smoothing to prevent overfitting

4. **Smoothing Formula**:
   ```
   Smoothed_Mean = (n × category_mean + m × global_mean) / (n + m)
   ```
   - n = number of samples in category
   - m = smoothing parameter (typically 5-20)

5. **Performance Benefits**:
   - Better R² scores compared to label encoding
   - Lower prediction errors (MAE, RMSE)
   - Captures meaningful category relationships

6. **When to Use**:
   - ✅ High cardinality categorical features
   - ✅ Features with no natural ordering
   - ✅ When target relationship exists
   - ❌ Very small datasets (overfitting risk)
   - ❌ Features with clear ordinal structure

### Remember
**Target Encoding is powerful but requires careful implementation to avoid data leakage and overfitting. Always use train-only encoding and consider smoothing!**